## Migration to AWS

The idea of this notebook is using torchtext on a manner that is compatible the migration to AWS. The following approach aims to 
avoid having to rely on the installation of torchtex, and actualizations of spicy on the network. 

In [1]:

%matplotlib inline
import numpy as np 
import pandas as pd 
import torch
import torchtext
from torchtext import data
import spacy
import os
import re

spacy.load('en')
_stopwords = spacy.lang.en.stop_words.STOP_WORDS

os.environ['OMP_NUM_THREADS'] = '4'


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(lower=True,include_lengths=False,tokenize='spacy')

LABEL = data.Field(sequential=False, 
                         use_vocab=False, 
                         pad_token=None, 
                            unk_token=None, dtype = torch.float)


dataFields = {"comment_text": ("comment_text", TEXT), 
              'toxic': ("toxic", LABEL), 
              'severe_toxic': ("severe_toxic", LABEL),
              'threat': ("threat", LABEL), 
              'obscene': ("obscene", LABEL),
              'insult': ("insult", LABEL), 
              'identity_hate': ("identity_hate", LABEL)}

dataset= data.TabularDataset(path='./data/train.json', 
                                            format='json',
                                            fields=dataFields, 
                                            skip_header=True)

In [2]:
import random
SEED = 3
#train, unimportant = dataset.split(split_ratio=0.5,random_state = random.seed(SEED)) 

train_data, val_data = dataset.split(split_ratio=0.9,random_state = random.seed(SEED))

In [3]:
MAX_VOCAB_SIZE = 20_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

In [4]:
import pickle

pickle.dump(TEXT, open('./custom_embeddings/train_data_field', 'wb'))

In [5]:
TEXT.vocab

In [6]:
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, val_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.comment_text),
    sort_within_batch = True,
    device = device)

# Translating to data loader

In [19]:
import json

In [84]:
def save_iterator(iterator, data_prefix, data_dir = './data', with_labels = True):

    yFields = ['toxic','severe_toxic',
               'obscene','threat','insult',
               'identity_hate']
 
    if with_labels:
        text_list =[]; labels_list=[] 
        for batch in iterator:
            text_list.append(batch.comment_text.tolist())
            labels = torch.stack([getattr(batch, y) for y in yFields])
            labels = torch.transpose(labels,0,1).tolist()
            labels_list.append(labels)
    
        with open(os.path.join(data_dir,
            data_prefix+'_labels_list.json'), 'w') as f:
            json.dump(labels_list, f)
        with open(os.path.join(data_dir,
            data_prefix+'_text_list.json'), 'w') as f:
            json.dump(text_list, f)
            
    else:
        text_list =[]
        for batch in iterator:
            text_list.append(batch.comment_text.tolist())

        with open(os.path.join(data_dir,
            data_prefix+'_text_list.json'), 'w') as f:
            json.dump(text_list, f)

In [85]:
save_iterator(train_iterator, 'train')
save_iterator(valid_iterator, 'val')

In [86]:
def _get_data(data_prefix, data_dir='./data', with_labels= True):
    #print("Get train data loader.")

    with open(os.path.join(data_dir,
              data_prefix+'_text_list.json'), 'r') as f:
        train_X = json.load(f)
    train_X = [torch.tensor(t) for t in train_X] #.long()
    
    if with_labels:
        with open(os.path.join(data_dir,
                  data_prefix+'_labels_list.json'), 'r') as f:
            train_y = json.load(f)

        train_y = [torch.tensor(t) for t in train_y] # .float().squeeze()
        return train_X , train_y
    
    else: 
        return train_X

In [87]:
class Data_iterator:
    def __init__(self, data_prefix, data_dir='./data'):
        self.X, self.y  = _get_data(data_prefix, data_dir)
        
    def __iter__(self):
        return iter(zip(self.X, self.y))

In [88]:
class Test_iterator:
    def __init__(self, data_prefix, data_dir='./data'):
        self.X = _get_data(data_prefix, data_dir, with_labels= False)
        
    def __iter__(self):
        return iter(self.X)

In [89]:
iterator_train = Data_iterator('train')
iterator_val = Data_iterator('val')

## Build model

In [90]:
import torch.nn as nn
from torch.functional import F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)


In [91]:
INPUT_DIM = len(TEXT.vocab) # 20002
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 6
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # 1

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, 
            FILTER_SIZES ,OUTPUT_DIM, DROPOUT, PAD_IDX)

In [92]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,092,306 trainable parameters


In [93]:
for i in train_iterator:
    aux=i
    break

from torchsummaryX import summary
print(aux.comment_text.size())
summary(model, aux.comment_text )

torch.Size([80, 256])
                      Kernel Shape       Output Shape   Params Mult-Adds
Layer                                                                   
0_embedding           [100, 20002]     [256, 80, 100]  2.0002M   2.0002M
1_convs.Conv2d_0  [1, 100, 2, 100]  [256, 100, 79, 1]    20.1k     1.58M
2_convs.Conv2d_1  [1, 100, 3, 100]  [256, 100, 78, 1]    30.1k     2.34M
3_convs.Conv2d_2  [1, 100, 4, 100]  [256, 100, 77, 1]    40.1k     3.08M
4_dropout                        -         [256, 300]        -         -
5_fc                      [300, 6]           [256, 6]   1.806k      1.8k
---------------------------------------------------------------------------
                         Totals
Total params          2.092306M
Trainable params      2.092306M
Non-trainable params        0.0
Mult-Adds                9.002M


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedding,"[100, 20002]","[256, 80, 100]",2000200.0,2000200.0
1_convs.Conv2d_0,"[1, 100, 2, 100]","[256, 100, 79, 1]",20100.0,1580000.0
2_convs.Conv2d_1,"[1, 100, 3, 100]","[256, 100, 78, 1]",30100.0,2340000.0
3_convs.Conv2d_2,"[1, 100, 4, 100]","[256, 100, 77, 1]",40100.0,3080000.0
4_dropout,-,"[256, 300]",NaN,NaN
5_fc,"[300, 6]","[256, 6]",1806.0,1800.0


In [94]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([20002, 100])


In [95]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.0361, -0.3528, -0.4494,  ..., -0.3391, -0.0521, -0.2626],
        [-0.8892,  0.3043,  0.9224,  ..., -0.2417, -0.1520,  0.0683],
        [ 0.2916,  0.0795, -0.0095,  ...,  0.3854,  0.3772, -1.5852]])

In [96]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.0361, -0.3528, -0.4494,  ..., -0.3391, -0.0521, -0.2626],
        [-0.8892,  0.3043,  0.9224,  ..., -0.2417, -0.1520,  0.0683],
        [ 0.2916,  0.0795, -0.0095,  ...,  0.3854,  0.3772, -1.5852]])


In [97]:
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-1.0361, -0.3528, -0.4494,  ..., -0.3391, -0.0521, -0.2626],
        [-0.8892,  0.3043,  0.9224,  ..., -0.2417, -0.1520,  0.0683],
        [ 0.2916,  0.0795, -0.0095,  ...,  0.3854,  0.3772, -1.5852]])


## Train our model

In [98]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [99]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [100]:
import numpy
from sklearn.metrics import roc_auc_score
def roc_auc(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    #rounded_preds = torch.sigmoid(preds)
    
    #assert preds.size()==y.size()
    
    #reds=rounded_preds.detach().numpy()

    #y=y.numpy()
    
    global var_y
    global var_preds
    var_y = y
    var_preds = preds

    acc = roc_auc_score(y, preds)

    
    return acc

In [101]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    preds_list=[]
    labels_list= []
 
    iterations=0
    for batch in iterator:
        iterations+=1
        
        batch_X, batch_y = batch
        
        optimizer.zero_grad()
        
        #text = batch_X
        
        predictions = model(batch_X).squeeze(1)
        
        #batch_labels=torch.stack([getattr(batch, y) for y in yFields]) #transpose?
        #batch_labels = torch.transpose(batch_labels,0,1)
        
        loss = criterion(predictions, batch_y)
        
        loss.backward()
        
        optimizer.step()
        
        preds_list+=[torch.sigmoid(predictions).detach().numpy()]
        labels_list+=[batch_y.numpy()]
        
        #if i%64==0:
        #    epoch_acc += [roc_auc(np.vstack(preds_list), np.vstack(batch_labels))]
        #    preds_list=[]
        #    labels_list= []
            
        
        epoch_loss += loss.item()
        
        
        
    return epoch_loss / iterations, roc_auc(np.vstack(preds_list), np.vstack(labels_list))

In [102]:


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    preds_list=[]
    labels_list= []
    epoch_acc=[]
    
    with torch.no_grad():
        iterations = 0
        for batch in iterator:
            iterations+=1
            
            batch_X, batch_y = batch
            
            predictions = model(batch_X).squeeze(1)
            
            #batch_labels = torch.stack([getattr(batch, y) for y in yFields]) #transpose?
            #batch_labels = torch.transpose(batch_labels,0,1)
            
            loss = criterion(predictions, batch_y)

            epoch_loss += loss.item()
            
            preds_list+=[torch.sigmoid(predictions).detach().numpy()]
            labels_list+=[batch_y.numpy()]
        
            #if i%64==0:
            #    epoch_acc += [roc_auc(np.vstack(preds_list), np.vstack(batch_labels))]
            #    preds_list=[]
            #    labels_list= []
        
    return epoch_loss / iterations, roc_auc(np.vstack(preds_list), np.vstack(labels_list))



In [103]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [104]:
model.embedding.weight.requires_grad = True


N_EPOCHS = 4

best_valid_loss = float('inf')


for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, iterator_train, optimizer, criterion)
    print('jaja')
    valid_loss, valid_acc = evaluate(model, iterator_val, criterion)
    print('juju')
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    

    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



jaja
juju
Epoch: 01 | Epoch Time: 3m 2s
	Train Loss: 0.080 | Train Acc: 89.20%
	 Val. Loss: 0.051 |  Val. Acc: 96.74%
jaja
juju
Epoch: 02 | Epoch Time: 3m 24s
	Train Loss: 0.054 | Train Acc: 96.62%
	 Val. Loss: 0.048 |  Val. Acc: 97.67%
jaja
juju
Epoch: 03 | Epoch Time: 3m 26s
	Train Loss: 0.048 | Train Acc: 97.66%
	 Val. Loss: 0.046 |  Val. Acc: 98.10%
jaja
juju
Epoch: 04 | Epoch Time: 3m 23s
	Train Loss: 0.045 | Train Acc: 98.13%
	 Val. Loss: 0.046 |  Val. Acc: 98.06%


In [105]:
TEXT.vocab.vectors = model.embedding.weight.data

In [106]:
import pickle

pickle.dump(TEXT, open('./custom_embeddings/train_data_field', 'wb'))

## Save word embedding

In [107]:
from tqdm import tqdm
word_list = [] 
def write_embeddings(path, embeddings, vocab):
    
    with open(path, 'w') as f:
        for i, embedding in enumerate(tqdm(embeddings)):
            word = vocab.itos[i]
            #skip words with unicode symbols
            if len(word) != len(word.encode()):
                continue
            word_list.append(word)
            vector = ' '.join([str(i) for i in embedding.tolist()])
            f.write(f'{word} {vector}\n')

In [108]:
write_embeddings('embeddings_conv.txt', 
                 model.embedding.weight.data, 
                 TEXT.vocab)

100%|██████████| 20002/20002 [00:02<00:00, 9894.67it/s]


## Testing 

In [111]:
from sklearn import metrics

#roc_auc(np.vstack(preds_list), np.vstack(labels_list))

In [112]:
dataFields = {"comment_text": ("comment_text", TEXT)}

testDataset= data.TabularDataset(path='./data/test.json', 
                                            format='json',
                                            fields=dataFields, 
                                            skip_header=False)

In [113]:
len(testDataset)

153164

In [114]:
test_iterator = torchtext.data.Iterator(testDataset, batch_size=64, device=device, 
                                     sort=False, sort_within_batch=False, 
                                     repeat=False,shuffle=False)

In [115]:
save_iterator(test_iterator, 'test', with_labels = False)

In [116]:
iterator_test = Test_iterator('test')

In [117]:
myPreds=[]
with torch.no_grad():
    model.eval()
    for batch_X in iterator_test:

        torch.cuda.empty_cache()
    
        predictions = model(batch_X).squeeze(1)         
        myPreds+=[torch.sigmoid(predictions).detach().numpy()]
    
        torch.cuda.empty_cache()
myPreds = np.vstack(myPreds)

In [118]:
testDF= pd.read_csv("./data/test.csv")
for i, col in enumerate(["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]):
    testDF[col] = myPreds[:, i]

In [119]:
myPreds.shape

(153164, 6)

In [120]:
testDF.drop("comment_text", axis=1).to_csv("submission_convolutional_no_torchtext.csv", index=False)